# Async Client

For high-throughput applications, use `AsyncESIOSClient` to make
concurrent API calls with `asyncio`.

In [1]:
import asyncio
from esios import AsyncESIOSClient

## Basic usage

The async client provides `get()` and `download()` methods that return
raw API responses (dicts), not DataFrames. You handle the data processing.

In [2]:
async def fetch_indicator(indicator_id: int) -> dict:
    async with AsyncESIOSClient() as client:
        data = await client.get(
            f"indicators/{indicator_id}",
            params={
                "start_date": "2025-01-01",
                "end_date": "2025-01-07",
            },
        )
        return data

result = await fetch_indicator(600)
print(f"Indicator: {result['indicator']['name']}")
print(f"Values: {len(result['indicator']['values'])}")

Indicator: Precio mercado SPOT Diario
Values: 3462


## Concurrent requests

Fetch multiple indicators in parallel:

In [3]:
async def fetch_multiple(ids: list[int]) -> list[dict]:
    async with AsyncESIOSClient() as client:
        tasks = [
            client.get(
                f"indicators/{iid}",
                params={
                    "start_date": "2025-01-01",
                    "end_date": "2025-01-07",
                },
            )
            for iid in ids
        ]
        return await asyncio.gather(*tasks)

results = await fetch_multiple([600, 612, 613, 614])
for r in results:
    ind = r["indicator"]
    print(f"  {ind['id']}: {ind['name']} — {len(ind['values'])} values")

  600: Precio mercado SPOT Diario — 3462 values
  612: Precio mercado SPOT Intradiario Sesión 1 — 1154 values
  613: Precio mercado SPOT Intradiario Sesión 2 — 1154 values
  614: Precio mercado SPOT Intradiario Sesión 3 — 576 values


## When to use async vs sync

| Use case | Client |
|----------|--------|
| Notebooks, scripts, CLI | `ESIOSClient` (sync) |
| Web apps (FastAPI, etc.) | `AsyncESIOSClient` |
| Bulk downloads (100+ requests) | `AsyncESIOSClient` |
| Simple analysis | `ESIOSClient` (sync) |

The sync client includes caching, DataFrame conversion, and geo pivoting.
The async client is lower-level — it returns raw API dicts.